In [ ]:
import numpy as np
import statsmodels.api as sm
from tqdm import tqdm
import matplotlib.pyplot as plt
import multiprocessing
from scipy import integrate
import pandas as pd
from helpers import confidence_intervals
import ray
ray.init()

In [ ]:
mc_preds = np.append(np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_2.npy'), 
                     np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_1.npy'), axis = 0)

In [ ]:
# read in val data
true_y = np.load('../../../../data/commaai/extracted_coefficients/copula_cil/Bzeta/labels_val.npy')

density_path = '../../../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

In [ ]:
densities = []
supports = []
for i in tqdm(range(0, mc_preds.shape[0])):
    kde = sm.nonparametric.KDEUnivariate(mc_preds[i,:,:])
    kde.fit() # Estimate the densities
    support = kde.support
    endog = kde.endog
    density = kde.density
    supports.append(support)
    densities.append(density) 

# get extended densities

In [ ]:
no_points = 750
grid = np.linspace(min(density['axes']), max(density['axes']), int(no_points))

In [ ]:
density_ext_list = []
density_ext = np.array(np.repeat(0, 750), dtype=float)
for j in tqdm(range(0, len(densities))):
    density_ext = np.array(np.repeat(0, 750), dtype=float)
    for i in range(0, supports[j].shape[0]):
        density_ext[np.where(np.abs(grid - supports[j][i]) == min(np.abs(grid - supports[j][i])))] = densities[j][i]
    density_ext_list.append(density_ext)

In [ ]:
density_ext_list = np.array(density_ext_list)
np.save('density_dropout.npy', density_ext_list)

In [ ]:
density_ext_list = np.load('density_dropout.npy')

In [ ]:
j = 0
cdf_mc_dropout = [] 
for supp in tqdm(density_ext_list):
    dens = density_ext_list[j,:]
    def f_integral(i):
        return(integrate.trapz(dens[i:(i+2)], grid[i:(i+2)]))
    with multiprocessing.Pool(20) as proc:
        probs = proc.map(f_integral, np.array([i for i in range(0, grid.shape[0])]))
    cdf_i = np.cumsum(np.array(probs))
    cdf_mc_dropout.append(cdf_i)
    j += 1

In [ ]:
np.save('../../../../data/commaai/predictions/mc_dropout/cdf_is_mc_dropout.npy', np.array(cdf_mc_dropout))

In [ ]:
x = np.load('../../../../data/commaai/predictions/mc_dropout/cdf_is_mc_dropout.npy')

In [ ]:
plt.scatter()

In [ ]:
alphas = np.linspace(0.05, 0.95, 10) #np.append(, float(0.99))
cis = confidence_intervals(alphas, cdf_mc_dropout, true_y, grid)

In [ ]:
levels = np.linspace(0.05, 0.95, 10) # np.append( , float(0.99)
plt.plot(levels, levels - np.array(cis['coverage_rate']), linestyle = '-.', label = 'MC-Dropout', color = 'red')
plt.xlabel((r'$1- \alpha$'))
plt.ylabel(r'$1- \alpha$ - coverage rate')

In [ ]:
np.array(cis['prediction_intervals']).shape